In [1]:
import glob
import time
import pandas as pd
# from xml.dom import minidom
from lxml import etree
from nltk import ngrams
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

In [2]:
%matplotlib inline

In [3]:

t1=time.time()
path='./dataset/blogs_train/'
all_files = glob.glob(path + "/*.xml")
li = []

for filename in all_files:
  # print("file name ", filename)
  li.append(filename)
    

t2=time.time()
print("time taken to read All csv :: ", t2-t1, " seconds ")

time taken to read All csv ::  0.03994870185852051  seconds 


In [4]:
def clean_file_names(li):
  # names_list=[]
  # for i in range(len(li)):
  name_list=li.split('/')[-1]
  return name_list

In [5]:
def get_attributes_from_name(name):
  # data_object=[]
  # for data in range(len(names_list)):
  #   dummy_var=names_list[data].split('.')
  #   dummy_object={'id':dummy_var[0], 'gender':dummy_var[1],
  #                 'age':dummy_var[2], 'category':dummy_var[3],
  #                 'horoscope':dummy_var[4]
  #                 }
  #   data_object.append(dummy_object)
  dummy_var=name.split('.')
  data_object={'id':dummy_var[0], 'gender':dummy_var[1],
                  'age':dummy_var[2], 'category':dummy_var[3],
                  'horoscope':dummy_var[4]
                }
  
  return data_object

In [6]:
parser = etree.XMLParser(encoding='ISO-8859-1', recover = True, huge_tree=True)

In [7]:
def read_xml(li):
  all_objects=[]
  for i in range(len(li)):
    # print("i will run ", i+1," times")
    name_of_file=clean_file_names(li[i])
    # print(name_of_file)
    data_object=get_attributes_from_name(name_of_file)
    # print(data_object)

    post_text='  '
    tree = etree.parse(li[i], parser=parser)
    root = tree.getroot()
    for element in root.iter():
        if element.tag == 'post':
            element_text = element.text
            post_text = post_text + element_text
    mystring = post_text.replace('\n', '').replace('\r', '').replace('\t', '')
    mystring = mystring.replace('   ', '').replace('\\','')
   


   
    # print(mystring)
    # name_of_file=clean_file_names(li[i])
    # data_object=get_attributes_from_name(name_of_file)
    



    data_object['posts']=mystring   
    all_objects.append(data_object)

  return all_objects



In [8]:
len(li)

14600

In [10]:
t1=time.time()
data=read_xml(li[0:])
t2=time.time()

In [11]:
print("time taken to read :: ", (t2-t1)/100)

time taken to read ::  0.14561341047286988


In [12]:
df=pd.DataFrame(data)

In [13]:
df.tail()

,id,gender,age,category,horoscope,posts
14595,3122872,male,15,Student,Libra,ok we got webspace off abp for bgs but of cour...
14596,3322392,female,15,indUnk,Pisces,"dearest diary, i do not know wad to put for t..."
14597,4008208,female,16,indUnk,Pisces,Good friends are the greatest treasures of th...
14598,1599685,male,23,Student,Capricorn,WE BRING THIS SPECIAL ANNOUCEMENT TO YOU LIVE...
14599,1608709,male,25,Accounting,Aries,Football Alex 'Knight of the Realm' Ferguson ...


In [14]:
data=''

## Multilabel classification

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [16]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

In [17]:
from sklearn.feature_extraction.text import CountVectorizer


In [18]:
sentences = df['posts'].values

In [19]:
labels=df['horoscope']

In [20]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(labels)

LabelEncoder()

In [21]:
le.classes_

array(['Aquarius', 'Aries', 'Cancer', 'Capricorn', 'Gemini', 'Leo',
       'Libra', 'Pisces', 'Sagittarius', 'Scorpio', 'Taurus', 'Virgo'],
      dtype=object)

In [22]:
y=le.transform(labels)

In [24]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb

max_features = 8000
# cut texts after this number of words (among top max_features most common words)
maxlen = 80
batch_size = 8




print('Loading data...')
x_train, x_test,y_train,  y_test = train_test_split(sentences,y, test_size=0.2, random_state=42)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

#### vectorize #######
vectorizer = CountVectorizer()
vectorizer.fit(x_train)

x_train = vectorizer.transform(x_train[:3000])
x_test  = vectorizer.transform(x_test[:1000])




Loading data...
11680 train sequences
2920 test sequences


In [ ]:

print('Pad sequences (samples x time)')
# x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
# x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(32, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(x_train[:3000], y_train[:3000],
          batch_size=8,
          epochs=5,
          validation_data=(x_test[:1000], y_test[:1000]))
print('Test score:', score)
print('Test accuracy:', acc)

Pad sequences (samples x time)
x_train shape: (3000, 696795)
x_test shape: (1000, 696795)
Build model...




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train...



Train on 3000 samples, validate on 1000 samples
Epoch 1/5







In [ ]:
(trainX, testX, trainY, testY) = train_test_split(sentences,y, test_size=0.2, random_state=42)

In [ ]:
x_test.shape

In [ ]:
vectorizer = CountVectorizer()
vectorizer.fit(trainX)

X_train = vectorizer.transform(trainX)
X_test  = vectorizer.transform(testX)

In [ ]:
from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.wrappers.scikit_learn import KerasClassifier